In [1]:
import sympy as se
# https://github.com/sympy/sympy/issues/2720

In [2]:
# x = 5 * y + z * y
# y = x - 5
# x = z / 1 + 5

In [3]:
x, y, z = se.symbols('x y z')

In [4]:
eq1 = se.Eq(x, y + z * y)
eq2 = se.Eq(y, x - 5)
eq3 = se.Eq(x, z * x**2)

In [5]:
se.pprint(eq1)

x = y⋅z + y


In [6]:
se.simplify([eq1, eq2, eq3])

[Eq(x, y*z + y), Eq(y, x - 5), Eq(x, x**2*z)]

### Heat flow (RC)

In [7]:
dT, Tin, Tout, R, Q_flow, C, Tprev, Qheat = se.symbols('dT,Tin,Tout,R,Q_flow,C,Tprev,Qheat', real=True)
# dT = Tin - Tout
eq1 = se.Eq(Tin - Tout, dT)
# dT = R * Q_flow
eq2 = se.Eq(R * Q_flow, dT)
# Q_flow = C * (Tin - Tprev) - Qheat
eq3 = se.Eq(C * (Tin - Tprev) - Qheat, -Q_flow)
#print(se.solve([eq1, eq2, eq3], ))

'''
duplicates = [dT]
i = 1
for v in duplicates:
    eq2 = eq2.subs(v, se.solve(eq1, v)[0])
#eq3 = se.Eq(Q_flow - Q1)
'''

# Replace dT
eq2 = eq2.subs(dT, se.solve(eq1, dT)[0])
# Replace dT
eq3 = eq3.subs(Q_flow, se.solve(eq2, Q_flow)[0])

se.pprint(se.solve(eq3, Tin)[0])

C⋅R⋅Tprev + Qheat⋅R + Tout
──────────────────────────
         C⋅R + 1          


### Heat flow (R2C2)

In [8]:
Tout, T1, dT1, R1, Qflow1, C1, Tp1, Qheat1, T2, dT2, R2, Qflow2, C2, Tp2, Qheat2, Qflow3, Qflow4 = \
    se.symbols('Tout,T1,dT1,R1,Qflow1,C1,Tp1,Qheat1,T2,dT2,R2,Qflow2,C2,Tp2,Qheat2,Qflow3,Qflow4', real=True)
# dT = Tin - Tout
eq1 = se.Eq(T1 - Tout, dT1)
# dT = R * Q_flow
eq2 = se.Eq(R1 * Qflow1, dT1)
# Q_flow = C * (Tin - Tprev) - Qheat
eq3 = se.Eq(C1 * (T1 - Tp1), -Qflow2)
# dT = Tin - Tout
eq4 = se.Eq(T2 - T1, dT2)
# dT = R * Q_flow
eq5 = se.Eq(R2 * Qflow3, dT2)
# Q_flow = C * (Tin - Tprev) - Qheat
eq6 = se.Eq(C2 * (T2 - Tp2), -Qflow4)
# Connect them
eq7 = se.Eq(Qflow3, Qflow4 + Qheat2)
eq8 = se.Eq(Qflow1, Qflow2 + Qflow3 + Qheat1)

# Replace dTs
eq2 = eq2.subs(dT1, se.solve(eq1, dT1)[0])
eq8 = eq8.subs(Qflow1, se.solve(eq2, Qflow1)[0])
eq5 = eq5.subs(dT2, se.solve(eq4, dT2)[0])
#eq7 = eq7.subs(Qflow3, se.solve(eq5, Qflow3)[0])

# For T1
eq9 = eq7.subs(Qflow3, se.solve(eq5, Qflow3)[0])
eq9 = eq9.subs(Qflow4, se.solve(eq6, Qflow4)[0])
print('T1')
se.pprint(se.solve(eq9, T1))

# Replace Cs
eq7 = eq7.subs(Qflow2, se.solve(eq3, Qflow2)[0])
eq7 = eq7.subs(Qflow4, se.solve(eq6, Qflow4)[0])
eq8 = eq8.subs(Qflow2, se.solve(eq3, Qflow2)[0])

# Initialize T1
eq8 = eq8.subs(T1, se.solve(eq5, T1)[0])

# Combine equations
eq8 = eq8.subs(Qflow3, se.solve(eq7, Qflow3)[0])

# R2C2
print('\nT2')
se.pprint(se.solve(eq8, T2))

T1
[C₂⋅R₂⋅T₂ - C₂⋅R₂⋅Tp₂ - Qheat₂⋅R₂ + T₂]

T2
⎡C₁⋅C₂⋅R₁⋅R₂⋅Tp₂ + C₁⋅Qheat₂⋅R₁⋅R₂ + C₁⋅R₁⋅Tp₁ + C₂⋅R₁⋅Tp₂ + C₂⋅R₂⋅Tp₂ + Qheat
⎢─────────────────────────────────────────────────────────────────────────────
⎣                                    C₁⋅C₂⋅R₁⋅R₂ + C₁⋅R₁ + C₂⋅R₁ + C₂⋅R₂ + 1  

₁⋅R₁ + Qheat₂⋅R₁ + Qheat₂⋅R₂ + Tout⎤
───────────────────────────────────⎥
                                   ⎦


### Heat flow (R4C2)

In [9]:
To, Ti, dTw2i, Rw2i, Qw2i, Ci, Ti_p, Qi_ext, Ts, dTs, Rs, Qis, Cs, Ts_p, Qs_ext, Qs, Qsc, Qi, Ris = \
    se.symbols('To,Ti,dTw2i,Rw2i,Qw2i,Ci,Ti_p,Qi_ext,Ts,dTs,Rs,Qis,Cs,Ts_p,Qs_ext,Qs,Qsc,Qi,Ris', real=True)
Qow1, Qw1w2, Qw1_ext, Tw1, dTow1, Qw2, Qw2i, Qw2_ext, Tw2, dTw1w2, Row1, Rw1w2, dTis = \
    se.symbols('Qow1,Qw1w2,Qw1_ext,Tw1,dTow1,Qw2,Qw2i,Qw2_ext,Tw2,dTw1w2,Row1, Rw1w2,dTis', real=True)

# Heat balance
eq1 = se.Eq(Qis, Qs + Qs_ext)
eq2 = se.Eq(Qw2i, Qi + Qis + Qi_ext)
eq3 = se.Eq(Qw1w2, Qw2i + Qw2_ext)
eq4 = se.Eq(Qow1, Qw1w2 + Qw1_ext)

# Resistive heat transfer
eq5 = se.Eq(Tw1 - To, dTow1)
eq6 = se.Eq(Row1 * Qow1, dTow1)
eq6 = eq6.subs(dTow1, se.solve(eq5, dTow1)[0])

eq7 = se.Eq(Tw2 - Tw1, dTw1w2)
eq8 = se.Eq(Rw1w2 * Qw1w2, dTw1w2)
eq8 = eq8.subs(dTw1w2, se.solve(eq7, dTw1w2)[0])

eq9 = se.Eq(Ti - Tw2, dTw2i)
eq10 = se.Eq(Rw2i * Qw2i, dTw2i)
eq10 = eq10.subs(dTw2i, se.solve(eq9, dTw2i)[0])

eq11 = se.Eq(Ts - Ti, dTis)
eq12 = se.Eq(Ris * Qis, dTis)
eq12 = eq12.subs(dTis, se.solve(eq11, dTis)[0])

# Heat storage
eq13 = se.Eq(Ci * (Ti - Ti_p), -Qi)
eq14 = se.Eq(Cs * (Ts - Ts_p), -Qs)

print('\nHeat Flow')
print(eq1)
print(eq2)
print(eq3)
print(eq4)
print('\nTemperatures')
print(eq6)
print(eq8)
print(eq10)
print(eq12)
print('\nCapacitors')
print(eq13)
print(eq14)


Heat Flow
Eq(Qis, Qs + Qs_ext)
Eq(Qw2i, Qi + Qi_ext + Qis)
Eq(Qw1w2, Qw2_ext + Qw2i)
Eq(Qow1, Qw1_ext + Qw1w2)

Temperatures
Eq(Qow1*Row1, -To + Tw1)
Eq(Qw1w2*Rw1w2, -Tw1 + Tw2)
Eq(Qw2i*Rw2i, Ti - Tw2)
Eq(Qis*Ris, -Ti + Ts)

Capacitors
Eq(Ci*(Ti - Ti_p), -Qi)
Eq(Cs*(Ts - Ts_p), -Qs)


In [10]:
eq41 = eq1.subs(Qs, se.solve(eq14, Qs)[0])
eq42 = eq2.subs(Qi, se.solve(eq13, Qi)[0])

print(eq41)
print(eq42)
print(eq3)
print(eq4)
print()
print(eq6)
print(eq8)
print(eq10)
print(eq12)

eq51 = eq41.subs(Qis, se.solve(eq12, Qis)[0])
eq52 = eq42.subs(Qw2i, se.solve(eq10, Qw2i)[0])
eq53 = eq3.subs(Qw1w2, se.solve(eq8, Qw1w2)[0])
eq54 = eq4.subs(Qow1, se.solve(eq6, Qow1)[0])
print()
print(eq51)
print(eq52)
print(eq53)
print(eq54)

eq61 = eq8.subs(Qw1w2, se.solve(eq54, Qw1w2)[0])
eq62 = eq10.subs(Qw2i, se.solve(eq53, Qw2i)[0])
eq63 = eq12.subs(Qis, se.solve(eq52, Qis)[0])

print('\n---NEXT---(Eliminated all Q; Only temperatures left)')
print(eq51)
print(eq61)
print(eq62)
print(eq63)

eq72 = eq62.subs(Ti, se.solve(eq51, Ti)[0])
eq73 = eq63.subs(Ti, se.solve(eq51, Ti)[0])

print('\n---NEXT---(Eliminated Ti)')
print(eq61)
print(eq72)
print(eq73)

eq81 = eq61.subs(Tw2, se.solve(eq73, Tw2)[0])
eq82 = eq72.subs(Tw2, se.solve(eq73, Tw2)[0])

print('\n---NEXT---(Eliminated Tw2)')
print(eq81)
print(eq82)

eq91 = eq81.subs(Tw1, se.solve(eq82, Tw1)[0])

print('\n---NEXT---(Eliminated Tw1)')
print(eq91)

print('\n---FINAL---')
print('Ts = ')
#se.pprint(se.solve(eq91, Ts))
print(se.solve(eq91, Ts)[0])

print('Ti = ')
#se.pprint(se.solve(eq51, Ti))
print(se.solve(eq51, Ti)[0])

Eq(Qis, Cs*(-Ts + Ts_p) + Qs_ext)
Eq(Qw2i, Ci*(-Ti + Ti_p) + Qi_ext + Qis)
Eq(Qw1w2, Qw2_ext + Qw2i)
Eq(Qow1, Qw1_ext + Qw1w2)

Eq(Qow1*Row1, -To + Tw1)
Eq(Qw1w2*Rw1w2, -Tw1 + Tw2)
Eq(Qw2i*Rw2i, Ti - Tw2)
Eq(Qis*Ris, -Ti + Ts)

Eq((-Ti + Ts)/Ris, Cs*(-Ts + Ts_p) + Qs_ext)
Eq((Ti - Tw2)/Rw2i, Ci*(-Ti + Ti_p) + Qi_ext + Qis)
Eq((-Tw1 + Tw2)/Rw1w2, Qw2_ext + Qw2i)
Eq((-To + Tw1)/Row1, Qw1_ext + Qw1w2)

---NEXT---(Eliminated all Q; Only temperatures left)
Eq((-Ti + Ts)/Ris, Cs*(-Ts + Ts_p) + Qs_ext)
Eq(Rw1w2*(-Qw1_ext*Row1 - To + Tw1)/Row1, -Tw1 + Tw2)
Eq(Rw2i*(-Qw2_ext*Rw1w2 - Tw1 + Tw2)/Rw1w2, Ti - Tw2)
Eq(Ris*(Rw2i*(Ci*Ti - Ci*Ti_p - Qi_ext) + Ti - Tw2)/Rw2i, -Ti + Ts)

---NEXT---(Eliminated Ti)
Eq(Rw1w2*(-Qw1_ext*Row1 - To + Tw1)/Row1, -Tw1 + Tw2)
Eq(Rw2i*(-Qw2_ext*Rw1w2 - Tw1 + Tw2)/Rw1w2, Cs*Ris*Ts - Cs*Ris*Ts_p - Qs_ext*Ris + Ts - Tw2)
Eq(Ris*(Cs*Ris*Ts - Cs*Ris*Ts_p - Qs_ext*Ris + Rw2i*(-Ci*Ti_p + Ci*(Cs*Ris*Ts - Cs*Ris*Ts_p - Qs_ext*Ris + Ts) - Qi_ext) + Ts - Tw2)/Rw2i, -Cs*Ris*Ts

### Heat flow (R5C3)

In [11]:
To, Ti, dTw2i, Rw2i, Qw2i, Ci, Ti_p, Qi_ext, Ts, dTs, Rs, Qis, Cs, Ts_p, Qs_ext, Qs, Qsc, Qi, Ris = \
    se.symbols('To,Ti,dTw2i,Rw2i,Qw2i,Ci,Ti_p,Qi_ext,Ts,dTs,Rs,Qis,Cs,Ts_p,Qs_ext,Qs,Qsc,Qi,Ris', real=True)
Qow1, Qw1w2, Qw1_ext, Tw1, dTow1, Qw2, Qw2i, Qw2_ext, Tw2, dTw1w2, Row1, Rw1w2, dTis = \
    se.symbols('Qow1,Qw1w2,Qw1_ext,Tw1,dTow1,Qw2,Qw2i,Qw2_ext,Tw2,dTw1w2,Row1, Rw1w2,dTis', real=True)
Qiw, Qw, Qw_ext, Tw, dTiw, Riw, Cw, Tw_p = \
    se.symbols('Qiw,Qw,Qw_ext,Tw,dTiw,Riw,Cw,Tw_p', real=True)

# Heat balance
eq0 = se.Eq(Qiw, Qw + Qw_ext)
eq1 = se.Eq(Qis, Qs + Qs_ext)
eq2 = se.Eq(Qw2i, Qi + Qis + Qiw +Qi_ext)
eq3 = se.Eq(Qw1w2, Qw2i + Qw2_ext)
eq4 = se.Eq(Qow1, Qw1w2 + Qw1_ext)

# Resistive heat transfer
eq5 = se.Eq(Tw1 - To, dTow1)
eq6 = se.Eq(Row1 * Qow1, dTow1)
eq6 = eq6.subs(dTow1, se.solve(eq5, dTow1)[0])

eq7 = se.Eq(Tw2 - Tw1, dTw1w2)
eq8 = se.Eq(Rw1w2 * Qw1w2, dTw1w2)
eq8 = eq8.subs(dTw1w2, se.solve(eq7, dTw1w2)[0])

eq9 = se.Eq(Ti - Tw2, dTw2i)
eq10 = se.Eq(Rw2i * Qw2i, dTw2i)
eq10 = eq10.subs(dTw2i, se.solve(eq9, dTw2i)[0])

eq11 = se.Eq(Ts - Ti, dTis)
eq12 = se.Eq(Ris * Qis, dTis)
eq12 = eq12.subs(dTis, se.solve(eq11, dTis)[0])

eq13 = se.Eq(Tw - Ti, dTiw)
eq14 = se.Eq(Riw * Qiw, dTiw)
eq14 = eq14.subs(dTiw, se.solve(eq13, dTiw)[0])

# Heat storage
eq15 = se.Eq(Ci * (Ti - Ti_p), -Qi)
eq16 = se.Eq(Cs * (Ts - Ts_p), -Qs)
eq17 = se.Eq(Cw * (Tw - Tw_p), -Qw)

print('\nHeat Flow')
print(eq0)
print(eq1)
print(eq2)
print(eq3)
print(eq4)
print('\nTemperatures')
print(eq6)
print(eq8)
print(eq10)
print(eq12)
print(eq14)
print('\nCapacitors')
print(eq15)
print(eq16)
print(eq17)


Heat Flow
Eq(Qiw, Qw + Qw_ext)
Eq(Qis, Qs + Qs_ext)
Eq(Qw2i, Qi + Qi_ext + Qis + Qiw)
Eq(Qw1w2, Qw2_ext + Qw2i)
Eq(Qow1, Qw1_ext + Qw1w2)

Temperatures
Eq(Qow1*Row1, -To + Tw1)
Eq(Qw1w2*Rw1w2, -Tw1 + Tw2)
Eq(Qw2i*Rw2i, Ti - Tw2)
Eq(Qis*Ris, -Ti + Ts)
Eq(Qiw*Riw, -Ti + Tw)

Capacitors
Eq(Ci*(Ti - Ti_p), -Qi)
Eq(Cs*(Ts - Ts_p), -Qs)
Eq(Cw*(Tw - Tw_p), -Qw)


In [12]:
eq41 = eq1.subs(Qs, se.solve(eq16, Qs)[0])
eq42 = eq2.subs(Qi, se.solve(eq15, Qi)[0])
eq43 = eq0.subs(Qw, se.solve(eq17, Qw)[0])
print(eq41)
print(eq42)
print(eq43)

eq51 = eq41.subs(Qis, se.solve(eq12, Qis)[0])
eq52 = eq42.subs(Qw2i, se.solve(eq10, Qw2i)[0])
eq53 = eq3.subs(Qw1w2, se.solve(eq8, Qw1w2)[0])
eq54 = eq4.subs(Qow1, se.solve(eq6, Qow1)[0])
eq55 = eq43.subs(Qiw, se.solve(eq14, Qiw)[0])
print()
print(eq51)
print(eq52)
print(eq53)
print(eq54)
print(eq55)

eq61 = eq8.subs(Qw1w2, se.solve(eq54, Qw1w2)[0])
eq62 = eq10.subs(Qw2i, se.solve(eq53, Qw2i)[0])
eq63 = eq12.subs(Qis, se.solve(eq52, Qis)[0]).subs(Qiw, se.solve(eq14, Qiw)[0]) #####
#eq64 = eq14.subs(Qiw, se.solve(eq52, Qiw)[0])
print('\n---NEXT---(Eliminated all Q; Only temperatures left)')
print(eq51)
print(eq61)
print(eq62)
print(eq63)
print(eq55)
#print(eq64)

eq72 = eq62.subs(Tw, se.solve(eq55, Tw)[0])
eq73 = eq63.subs(Tw, se.solve(eq55, Tw)[0])
print('\n---NEXT---(Eliminated Tw)')
print(eq51)
print(eq61)
print(eq72)
print(eq73)

eq82 = eq72.subs(Ti, se.solve(eq51, Ti)[0])
eq83 = eq73.subs(Ti, se.solve(eq51, Ti)[0])
print('\n---NEXT---(Eliminated Ti)')
print(eq61)
print(eq82)
print(eq83)

eq91 = eq61.subs(Tw2, se.solve(eq83, Tw2)[0])
eq92 = eq82.subs(Tw2, se.solve(eq83, Tw2)[0])
print('\n---NEXT---(Eliminated Tw2)')
print(eq91)
print(eq92)

eq101 = eq91.subs(Tw1, se.solve(eq92, Tw1)[0])
print('\n---NEXT---(Eliminated Tw1)')
print(eq101)

print('\n---FINAL---')
print('Ts = ')
#se.pprint(se.solve(eq91, Ts))
print(se.solve(eq101, Ts)[0])

print('Ti = ')
#se.pprint(se.solve(eq51, Ti))
print(se.solve(eq51, Ti)[0])

print('Tw = ')
#se.pprint(se.solve(eq51, Ti))
print(se.solve(eq55, Tw)[0])

Eq(Qis, Cs*(-Ts + Ts_p) + Qs_ext)
Eq(Qw2i, Ci*(-Ti + Ti_p) + Qi_ext + Qis + Qiw)
Eq(Qiw, Cw*(-Tw + Tw_p) + Qw_ext)

Eq((-Ti + Ts)/Ris, Cs*(-Ts + Ts_p) + Qs_ext)
Eq((Ti - Tw2)/Rw2i, Ci*(-Ti + Ti_p) + Qi_ext + Qis + Qiw)
Eq((-Tw1 + Tw2)/Rw1w2, Qw2_ext + Qw2i)
Eq((-To + Tw1)/Row1, Qw1_ext + Qw1w2)
Eq((-Ti + Tw)/Riw, Cw*(-Tw + Tw_p) + Qw_ext)

---NEXT---(Eliminated all Q; Only temperatures left)
Eq((-Ti + Ts)/Ris, Cs*(-Ts + Ts_p) + Qs_ext)
Eq(Rw1w2*(-Qw1_ext*Row1 - To + Tw1)/Row1, -Tw1 + Tw2)
Eq(Rw2i*(-Qw2_ext*Rw1w2 - Tw1 + Tw2)/Rw1w2, Ti - Tw2)
Eq(Ris*(Rw2i*(Ci*Ti - Ci*Ti_p - Qi_ext - (-Ti + Tw)/Riw) + Ti - Tw2)/Rw2i, -Ti + Ts)
Eq((-Ti + Tw)/Riw, Cw*(-Tw + Tw_p) + Qw_ext)

---NEXT---(Eliminated Tw)
Eq((-Ti + Ts)/Ris, Cs*(-Ts + Ts_p) + Qs_ext)
Eq(Rw1w2*(-Qw1_ext*Row1 - To + Tw1)/Row1, -Tw1 + Tw2)
Eq(Rw2i*(-Qw2_ext*Rw1w2 - Tw1 + Tw2)/Rw1w2, Ti - Tw2)
Eq(Ris*(Rw2i*(Ci*Ti - Ci*Ti_p - Qi_ext - (-Ti + (Cw*Riw*Tw_p + Qw_ext*Riw + Ti)/(Cw*Riw + 1))/Riw) + Ti - Tw2)/Rw2i, -Ti + Ts)

---NEXT---(Eli

# Heatflow (R6C3)

In [13]:
To, Ti, dTw2i, Rw2i, Qw2i, Ci, Ti_p, Qi_ext, Ts, dTs, Rs, Qis, Cs, Ts_p, Qs_ext, Qs, Qsc, Qi, Ris = \
    se.symbols('To,Ti,dTw2i,Rw2i,Qw2i,Ci,Ti_p,Qi_ext,Ts,dTs,Rs,Qis,Cs,Ts_p,Qs_ext,Qs,Qsc,Qi,Ris', real=True)
Qow1, Qw1w2, Qw1_ext, Tw1, dTow1, Qw2, Qw2i, Qw2_ext, Tw2, dTw1w2, Row1, Rw1w2, dTis = \
    se.symbols('Qow1,Qw1w2,Qw1_ext,Tw1,dTow1,Qw2,Qw2i,Qw2_ext,Tw2,dTw1w2,Row1, Rw1w2,dTis', real=True)
Qiw, Qw, Qw_ext, Tw, dTiw, Riw, Cw, Tw_p, Roi, Qoi, dToi = \
    se.symbols('Qiw,Qw,Qw_ext,Tw,dTiw,Riw,Cw,Tw_p,Roi,Qoi,dToi', real=True)

# Heat balance
eq0 = se.Eq(Qiw, Qw + Qw_ext)
eq1 = se.Eq(Qis, Qs + Qs_ext)
eq2 = se.Eq(Qw2i, Qi + Qis + Qiw + Qi_ext - Qoi)
eq3 = se.Eq(Qw1w2, Qw2i + Qw2_ext)
eq4 = se.Eq(Qow1, Qw1w2 + Qw1_ext)

# Resistive heat transfer
eq5 = se.Eq(Tw1 - To, dTow1)
eq6 = se.Eq(Row1 * Qow1, dTow1)
eq6 = eq6.subs(dTow1, se.solve(eq5, dTow1)[0])

eq7 = se.Eq(Tw2 - Tw1, dTw1w2)
eq8 = se.Eq(Rw1w2 * Qw1w2, dTw1w2)
eq8 = eq8.subs(dTw1w2, se.solve(eq7, dTw1w2)[0])

eq9 = se.Eq(Ti - Tw2, dTw2i)
eq10 = se.Eq(Rw2i * Qw2i, dTw2i)
eq10 = eq10.subs(dTw2i, se.solve(eq9, dTw2i)[0])

eq11 = se.Eq(Ts - Ti, dTis)
eq12 = se.Eq(Ris * Qis, dTis)
eq12 = eq12.subs(dTis, se.solve(eq11, dTis)[0])

eq13 = se.Eq(Tw - Ti, dTiw)
eq14 = se.Eq(Riw * Qiw, dTiw)
eq14 = eq14.subs(dTiw, se.solve(eq13, dTiw)[0])

eq18 = se.Eq(Ti - To, dToi)
eq19 = se.Eq(Roi * Qoi, dToi)
eq19 = eq19.subs(dToi, se.solve(eq18, dToi)[0])

# Heat storage
eq15 = se.Eq(Ci * (Ti - Ti_p), -Qi)
eq16 = se.Eq(Cs * (Ts - Ts_p), -Qs)
eq17 = se.Eq(Cw * (Tw - Tw_p), -Qw)

print('\nHeat Flow')
print(eq0)
print(eq1)
print(eq2)
print(eq3)
print(eq4)
print('\nTemperatures')
print(eq6)
print(eq8)
print(eq10)
print(eq12)
print(eq14)
print(eq19)
print('\nCapacitors')
print(eq15)
print(eq16)
print(eq17)


Heat Flow
Eq(Qiw, Qw + Qw_ext)
Eq(Qis, Qs + Qs_ext)
Eq(Qw2i, Qi + Qi_ext + Qis + Qiw - Qoi)
Eq(Qw1w2, Qw2_ext + Qw2i)
Eq(Qow1, Qw1_ext + Qw1w2)

Temperatures
Eq(Qow1*Row1, -To + Tw1)
Eq(Qw1w2*Rw1w2, -Tw1 + Tw2)
Eq(Qw2i*Rw2i, Ti - Tw2)
Eq(Qis*Ris, -Ti + Ts)
Eq(Qiw*Riw, -Ti + Tw)
Eq(Qoi*Roi, Ti - To)

Capacitors
Eq(Ci*(Ti - Ti_p), -Qi)
Eq(Cs*(Ts - Ts_p), -Qs)
Eq(Cw*(Tw - Tw_p), -Qw)


In [14]:
eq41 = eq1.subs(Qs, se.solve(eq16, Qs)[0])
eq42 = eq2.subs(Qi, se.solve(eq15, Qi)[0])
eq43 = eq0.subs(Qw, se.solve(eq17, Qw)[0])
print(eq41)
print(eq42)
print(eq43)

eq51 = eq41.subs(Qis, se.solve(eq12, Qis)[0])
eq52 = eq42.subs(Qw2i, se.solve(eq10, Qw2i)[0]).subs(Qoi, se.solve(eq19, Qoi)[0])
eq53 = eq3.subs(Qw1w2, se.solve(eq8, Qw1w2)[0])
eq54 = eq4.subs(Qow1, se.solve(eq6, Qow1)[0])
eq55 = eq43.subs(Qiw, se.solve(eq14, Qiw)[0])
print()
print(eq51)
print(eq52)
print(eq53)
print(eq54)
print(eq55)

eq61 = eq8.subs(Qw1w2, se.solve(eq54, Qw1w2)[0])
eq62 = eq10.subs(Qw2i, se.solve(eq53, Qw2i)[0])
eq63 = eq12.subs(Qis, se.solve(eq52, Qis)[0]).subs(Qiw, se.solve(eq14, Qiw)[0]) #####
#eq64 = eq14.subs(Qiw, se.solve(eq52, Qiw)[0])
print('\n---NEXT---(Eliminated all Q; Only temperatures left)')
print(eq51)
print(eq61)
print(eq62)
print(eq63)
print(eq55)
#print(eq64)

eq72 = eq62.subs(Tw, se.solve(eq55, Tw)[0])
eq73 = eq63.subs(Tw, se.solve(eq55, Tw)[0])
print('\n---NEXT---(Eliminated Tw)')
print(eq51)
print(eq61)
print(eq72)
print(eq73)

eq82 = eq72.subs(Ti, se.solve(eq51, Ti)[0])
eq83 = eq73.subs(Ti, se.solve(eq51, Ti)[0])
print('\n---NEXT---(Eliminated Ti)')
print(eq61)
print(eq82)
print(eq83)

eq91 = eq61.subs(Tw2, se.solve(eq83, Tw2)[0])
eq92 = eq82.subs(Tw2, se.solve(eq83, Tw2)[0])
print('\n---NEXT---(Eliminated Tw2)')
print(eq91)
print(eq92)

eq101 = eq91.subs(Tw1, se.solve(eq92, Tw1)[0])
print('\n---NEXT---(Eliminated Tw1)')
print(eq101)

print('\n---FINAL---')
print('Ts = ')
#se.pprint(se.solve(eq91, Ts))
print(se.solve(eq101, Ts)[0])

print('Ti = ')
#se.pprint(se.solve(eq51, Ti))
print(se.solve(eq51, Ti)[0])

print('Tw = ')
#se.pprint(se.solve(eq51, Ti))
print(se.solve(eq55, Tw)[0])

Eq(Qis, Cs*(-Ts + Ts_p) + Qs_ext)
Eq(Qw2i, Ci*(-Ti + Ti_p) + Qi_ext + Qis + Qiw - Qoi)
Eq(Qiw, Cw*(-Tw + Tw_p) + Qw_ext)

Eq((-Ti + Ts)/Ris, Cs*(-Ts + Ts_p) + Qs_ext)
Eq((Ti - Tw2)/Rw2i, Ci*(-Ti + Ti_p) + Qi_ext + Qis + Qiw - (Ti - To)/Roi)
Eq((-Tw1 + Tw2)/Rw1w2, Qw2_ext + Qw2i)
Eq((-To + Tw1)/Row1, Qw1_ext + Qw1w2)
Eq((-Ti + Tw)/Riw, Cw*(-Tw + Tw_p) + Qw_ext)

---NEXT---(Eliminated all Q; Only temperatures left)
Eq((-Ti + Ts)/Ris, Cs*(-Ts + Ts_p) + Qs_ext)
Eq(Rw1w2*(-Qw1_ext*Row1 - To + Tw1)/Row1, -Tw1 + Tw2)
Eq(Rw2i*(-Qw2_ext*Rw1w2 - Tw1 + Tw2)/Rw1w2, Ti - Tw2)
Eq(Ris*(Ci*Ti - Ci*Ti_p - Qi_ext + Ti/Rw2i - Tw2/Rw2i + Ti/Roi - To/Roi - (-Ti + Tw)/Riw), -Ti + Ts)
Eq((-Ti + Tw)/Riw, Cw*(-Tw + Tw_p) + Qw_ext)

---NEXT---(Eliminated Tw)
Eq((-Ti + Ts)/Ris, Cs*(-Ts + Ts_p) + Qs_ext)
Eq(Rw1w2*(-Qw1_ext*Row1 - To + Tw1)/Row1, -Tw1 + Tw2)
Eq(Rw2i*(-Qw2_ext*Rw1w2 - Tw1 + Tw2)/Rw1w2, Ti - Tw2)
Eq(Ris*(Ci*Ti - Ci*Ti_p - Qi_ext + Ti/Rw2i - Tw2/Rw2i + Ti/Roi - To/Roi - (-Ti + (Cw*Riw*Tw_p + Qw_ext*R

(Ci*Cs*Cw*Ris*Riw*Roi*Row1*Ts_p + Ci*Cs*Cw*Ris*Riw*Roi*Rw1w2*Ts_p + Ci*Cs*Cw*Ris*Riw*Roi*Rw2i*Ts_p + Ci*Cs*Ris*Roi*Row1*Ts_p + Ci*Cs*Ris*Roi*Rw1w2*Ts_p + Ci*Cs*Ris*Roi*Rw2i*Ts_p + Ci*Cw*Qs_ext*Ris*Riw*Roi*Row1 + Ci*Cw*Qs_ext*Ris*Riw*Roi*Rw1w2 + Ci*Cw*Qs_ext*Ris*Riw*Roi*Rw2i + Ci*Cw*Riw*Roi*Row1*Ti_p + Ci*Cw*Riw*Roi*Rw1w2*Ti_p + Ci*Cw*Riw*Roi*Rw2i*Ti_p + Ci*Qs_ext*Ris*Roi*Row1 + Ci*Qs_ext*Ris*Roi*Rw1w2 + Ci*Qs_ext*Ris*Roi*Rw2i + Ci*Roi*Row1*Ti_p + Ci*Roi*Rw1w2*Ti_p + Ci*Roi*Rw2i*Ti_p + Cs*Cw*Ris*Riw*Roi*Ts_p + Cs*Cw*Ris*Riw*Row1*Ts_p + Cs*Cw*Ris*Riw*Rw1w2*Ts_p + Cs*Cw*Ris*Riw*Rw2i*Ts_p + Cs*Cw*Ris*Roi*Row1*Ts_p + Cs*Cw*Ris*Roi*Rw1w2*Ts_p + Cs*Cw*Ris*Roi*Rw2i*Ts_p + Cs*Cw*Riw*Roi*Row1*Ts_p + Cs*Cw*Riw*Roi*Rw1w2*Ts_p + Cs*Cw*Riw*Roi*Rw2i*Ts_p + Cs*Ris*Roi*Ts_p + Cs*Ris*Row1*Ts_p + Cs*Ris*Rw1w2*Ts_p + Cs*Ris*Rw2i*Ts_p + Cs*Roi*Row1*Ts_p + Cs*Roi*Rw1w2*Ts_p + Cs*Roi*Rw2i*Ts_p + Cw*Qi_ext*Riw*Roi*Row1 + Cw*Qi_ext*Riw*Roi*Rw1w2 + Cw*Qi_ext*Riw*Roi*Rw2i + Cw*Qs_ext*Ris*Riw*Roi + Cw*Qs_ext*Ris*

### Get Qw2i

In [36]:
print(eq3)
print(eq4)
print(eq6)
print(eq8)
print(eq10)

Eq(Qw1w2, Qw2_ext + Qw2i)
Eq(Qow1, Qw1_ext + Qw1w2)
Eq(Qow1*Row1, -To + Tw1)
Eq(Qw1w2*Rw1w2, -Tw1 + Tw2)
Eq(Qw2i*Rw2i, Ti - Tw2)


In [35]:
eq_w1 = eq3.subs(Qw1w2, se.solve(eq4, Qw1w2)[0])
eq_w2 = eq_w1.subs(Qow1, se.solve(eq6, Qow1)[0]).subs(Qw2i, se.solve(eq10, Qw2i)[0]).subs(Tw1, se.solve(eq8, Tw1)[0])
eq_w3 = eq_w2.subs(Tw2, se.solve(eq10, Tw2)[0]).subs(Qw1w2, se.solve(eq3, Qw1w2)[0])
print('Qw2i = ', se.solve(eq_w3, Qw2i)[0])

Qw2i =  (-Qw1_ext*Row1 - Qw2_ext*Row1 - Qw2_ext*Rw1w2 + Ti - To)/(Row1 + Rw1w2 + Rw2i)
